In [ ]:
#see https://github.com/fukunagatsu/CapR for more details

In [ ]:
!git clone https://github.com/fukunagatsu/CapR.git

In [ ]:
import subprocess
import pandas as pd
import os

In [ ]:
os.makedirs('./temp_fasta', exist_ok=True)

In [ ]:
def save_fasta(filename, seq):
    header = '>test\n'
    
    with open(f'./temp_fasta/{filename}.fasta', 'w') as file:
        file.write(header + seq)

In [ ]:
def capr(chunks):
    for id, seq in tqdm(chunks.values):
        capr_executable = './CapR/CapR'

        save_fasta(id, seq)
        
        # If CapR requires command line arguments, add them here
        args = [f'./temp_fasta/{id}.fasta', f'./CapR/CapR_predictions/{id}.txt', str(len(seq))]
        
        # Combine the executable and the arguments in a single command
        command = [capr_executable] + args
        
        # Run the command
        process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Capture the output and errors
        output = process.stdout.decode()
        errors = process.stderr.decode()
        print(errors)

        os.remove(f'./temp_fasta/{id}.fasta')

In [ ]:
import shutil
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count

def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [ ]:
train_df = pd.read_parquet('train_data_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(capr)(x)
    for i,x in enumerate(split_dataframe(train_df[['bpp_path', 'sequence']], 10000))
)
del train_df

In [ ]:
test_df = pd.read_parquet('test_sequences_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(capr)(x)
    for i,x in enumerate(split_dataframe(test_df[['bpp_path', 'sequence']], 10000))
)
del test_df